In [ ]:
#generate input random 1
import random
[n, m]=[int(x) for x in input().split()]
Q=[[0 for i in range (m)] for j in range (n)]
d=[[[0, [j]] for i in range (m+1)] for j in range (m+1)]
for i in range (n):
  for j in range (m):
    Q[i][j]=random.randint(1,4)
for i in range (m+1):
  for j in range (m+1):
    if (i==j): d[i][j][0]=0
    else:
      d[i][j][0]=random.randint(2,9)
for i in range(n):
  print(*Q[i])
for i in range(m+1):
  for j in range(m+1):
    print(d[i][j][0], end=' ')
  print()
q=[10 for i in range (n)]
print(*q)

5 5
4 4 4 3 1
1 3 3 3 4
2 1 2 3 4
3 2 1 4 1
4 3 3 2 3
0 3 2 9 4 4 
3 0 5 3 4 5 
6 9 0 2 2 9 
5 3 5 0 2 5 
9 3 4 6 0 7 
7 5 8 7 7 0 
10 10 10 10 10


In [ ]:
#generate input random 2
import random

while True:
    n = 100
    m = 50
    Q = [[random.randint(1, 1000) for j in range(m)] for i in range(n)]
    d = [[0 if i==j else random.randint(1, 1000) for j in range(m+1)] for i in range(m+1)]
    q = [300*m for i in range(n)]
    check = True
    for i in range(n):
        if sum(Q[i]) < q[i]:
            check = False
    if check:
        break

print(n, m)
for i in range(n):
    print(*Q[i])
for i in range(m+1):
    print(*d[i])
print(*q)

In [ ]:
#input
def inp():
    [n, m] = [int(x) for x in input().split()]
    Q = [[int(x) for x in input().split()] for i in range(n)]
    d = [[[int(x), [i]] for x in input().split()] for i in range(m+1)]
    q = [int(x) for x in input().split()]
    return n, m, Q, d, q

n, m, Q, d, q = inp()

In [ ]:
#fast fill in example
m=3
n=4
#Q=[[3,0,0,0],[0,2,0,1],[0,0,3,1]]
Q=[[3,0,0],[0,2,0],[0,0,3],[0,1,1]]
d=[[0,3,4,5],[3,0,3,2],[4,3,0,6],[5,2,6,0]]
q=[2,1,2,1]

In [ ]:
#input from file
with open('input.txt') as f:
    al=f.readlines()
# print(al)
(m, n) = al[0].strip().split(' ')
m=int(m)
n=int(n)
Q=[[]for i in range(n)]
for i in range(n):
    k = al[i+1].strip('\n').split(' ')
    for j in range (m):
        Q[i].append(int(k[j]))
print(Q)
d=[[]for i in range(m+1)]
for i in range(m+1):
    k = al[i+n+1].strip('\n').split(' ')
    for j in range(m+1):
        d[i].append([int(k[j]),[i]])
q=[]
print(d)
k = al[m+n+2].strip('\n').split(' ')
for i in range(n):
    q.append(int(k[i]))
print(q)

In [ ]:
#pre-processing

def preprocess():
    #the preprocess path turn the objective solution into a path in which each self is visited once
    #using Floyd Warshall algorithm to find shortest path between any 2 shelves
    for k in range(m+1):
        for i in range(m+1):
            for j in range(m+1):
                if d[i][j][0] > d[i][k][0] + d[k][j][0]:
                    d[i][j][0] = d[i][k][0] + d[k][j][0]
                    d[i][j][1] = d[i][k][1] + d[k][j][1]
preprocess()


In [ ]:
f = 0
fmin = 10**10
path = []
pathmin = []
visit = [0 for i in range(m+1)]
bt = [0 for i in range(m+1)]
p = [0 for i in range(n)]

def outp():
    global fmin, pathmin
    print(*pathmin)
    print(fmin)

dist = [10**10 for i in range(n)]

def bound():
    global dist, Q
    for k in range(n):
        for j in range(1, m+1):
            for i in range(m+1):
                if i != j:
                    dist[k] = min(dist[k], d[i][j][0]/Q[k][j-1])

def sol(last):
    global f, fmin, p, path, pathmin
    for i in range(n):
        if p[i] < q[i]:
            return False
    if f + d[last][0][0] < fmin:
        fmin = f + d[last][0][0]
        pathmin = path + d[last][0][1] + [0]
    return True

def backtrack(x,last):
    global f, path, visit, bt, p, fmin, q, dist
    if sol(last):
        return
    for i in range(1, m+1):
        if visit[i] == 0:
            if f + d[last][i][0] > fmin:
                continue
            check = False
            for j in range(n):
                if f + d[last][i][0] + dist[j]*max(0, q[j]-p[j]-Q[j][i-1]) > fmin:
                    check = True
                    break
            if check:
                continue
            bt[x] = i
            f += d[last][i][0]
            path += d[last][i][1]
            visit[i] = 1
            for j in range(n):
                p[j] += Q[j][i-1]
            backtrack(x+1, i)
            for j in range(n):
                p[j] -= Q[j][i-1]
            visit[i] = 0
            for j in range(len(d[last][i][1])):
                path.pop()
            f -= d[last][i][0]

bound()
backtrack(0,0)
outp()

0 2 4 1 5 0
19


In [ ]:
#dynamic programming

inf = 10**10

DP = {}
next = {}

def dp(shelves, last, goods, left):
    if (shelves, last, tuple(goods), left) in DP:
        return DP[(shelves, last, tuple(goods), left)]
    if left <= 0:
        next[(shelves, last, tuple(goods), left)] = (0,0,0,0)
        return d[last][0][0]
    if shelves == 1 << m:
        return inf
    bin = 1
    res = inf
    check = False
    for j in range(1, m+1):
        if shelves&bin == 0:         
            Goods = goods[:]            
            Left = left
            for i in range(n):
                Goods[i] += Q[i][j-1]
                if goods[i] < q[i] and Q[i][j-1] > 0:
                    check = True                    
                    if Goods[i] >= q[i]:
                        Left -= 1
            if check:
                x = dp(shelves|bin, j, Goods, Left) + d[last][j][0]
                if x < res:
                    res = x
                    next[(shelves, last, tuple(goods), left)] = (shelves|bin, j, tuple(Goods), Left)
        bin <<= 1
    DP[(shelves, last, tuple(goods), left)] = res
    return DP[(shelves, last, tuple(goods), left)]

def outp():
    x = dp(0, 0, [0]*n, n)
    i = (0, 0, tuple([0]*n), n)
    while i != (0,0,0,0):
        print(*d[i[1]][next[i][1]][1], end=' ')
        i = next[i]
    print(0)
    print(x)

preprocess()
outp()

0 2 4 1 5 0
19


In [ ]:
#greedy algorithm 1

def check(sum):
  for i in range (len(sum)):
    if sum[i]<q[i]:
      return False
  return True

visit=[True for i in range (n)]
sum=[0 for i in range (n)]
remain=[]
for i in range(1,m+1):
  remain.append(i)
start=0
path=[0]
pathcost=0
while (check(sum)==False and len(remain)!=0):
  local_min=d[start][remain[0]][0]
  start=remain[0]
  for i in range (1,len(remain)):
    if d[start][remain[i]][0] < local_min:
      local_min = d[start][remain[i]][0]
      start=remain[i]
  pathcost+=local_min
  remain.remove(start)
  for i in range (n): #len of sum
    sum[i]+=Q[i][start-1]
  path.append(start)
path.append(0)
pathcost+=d[start][0][0]
print(*path)
print(pathcost)


0 1 3 4 2 5 0
25


In [ ]:
#2nd greedy algorithm

ans = 0
Path = []

def greedy():
    global d, q, Q, ans, Path
    left = n
    visited = [False for i in range(m+1)]
    cur = 0
    picked = [0 for j in range(n)]
    while left > 0:    
        Max = 0
        for i in range(1, m+1):
            if visited[i]:
                continue
            goods = 0
            for j in range(n):
                goods += max(0, min(q[j]-picked[j], Q[j][i-1]))
            if goods/d[cur][i][0] > Max:
                Max = goods/d[cur][i][0]
                next = i
        for j in range(n):
            if picked[j] < q[j] and picked[j]+Q[j][next-1] >= q[j]:
                left -= 1
            picked[j] += Q[j][next-1]    
        ans += d[cur][next][0]
        cur = next
        Path.append(cur)
        visited[cur] = True
    ans += d[cur][0][0]

def outp():
    global Path, d, ans
    print(*d[0][Path[0]][1], end=' ')
    for i in range(len(Path)-1):
        print(*d[Path[i]][Path[i+1]][1], end=' ')
    print(*d[Path[len(Path)-1]][0][1], end=' ')
    print(0)
    print(ans)

greedy()
outp()

0 2 4 1 3 5 0
22


In [ ]:
#variable neighborhood search

import random
import math

curdist = 0
curpath = [0]
curgoods = [0]*n
curleft = n
curpool = []

bestdist = 10**10
bestpath = []

def outp():
    global bestpath, d, bestdist 
    for i in range(len(bestpath)-1):
        print(*d[bestpath[i]][bestpath[i+1]][1], end=' ')
    print(0)
    print(bestdist)

full = [i for i in range(1, m+1)]
random.shuffle(full)
full *= 2
r = 0
last_r = 0

def initial():
    # global curpath, curdist, d
    # curpath = [i for i in range(1, m+1)]
    # random.shuffle(curpath)
    # curpath = [0] + curpath + [0]
    # for i in range(len(curpath)-1):
    #     curdist += d[curpath[i]][curpath[i+1]][0]

    global full, r, curleft, n, curgoods, Q, curpath, curdist, curpool, m, last_r
    curdist = 0
    curpath = [0]
    curgoods = [0]*n
    curleft = n
    ir = 0
    while curleft > 0:
        for j in range(n):
            if curgoods[j] < q[j] and curgoods[j] + Q[j][full[r]-1] >= q[j]:
                curleft -= 1
            curgoods[j] += Q[j][full[r]-1]
        curdist += d[curpath[ir]][full[r]][0]
        curpath.append(full[r])
        r += 1
        ir += 1
    curdist += d[curpath[ir]][0][0]
    curpath += [0]
    curpool = full[:last_r] + full[r:m] if r < m else full[r:last_r+m]
    last_r = r

def cut(path, dist, goods, left, pool, num):
    global d, q, Q
    ret = dist
    ret_path = []
    ret_goods = []
    ret_left = left
    ret_pool = []
    for i1 in range(1, len(path)-1):
        x1 = path[i1]
        for k in range(n):
            if goods[k] >= q[k] and goods[k]-Q[k][x1-1] < q[k]:
                left += 1
            goods[k] -= Q[k][x1-1]
        dist += d[path[i1-1]][path[i1+1]][0] \
              - d[path[i1-1]][x1][0] - d[x1][path[i1+1]][0]
        path = path[:i1] + path[i1+1:]
        pool.append(x1)

        if left <= 0 and dist < ret:
            ret = dist
            ret_path = path[:]
            ret_goods = goods[:]
            ret_left = left
            ret_pool = pool[:]
        
        if num >= 2:
            for i2 in range(1, len(path)-1):
                x2 = path[i2]
                for k in range(n):
                    if goods[k] >= q[k] and goods[k]-Q[k][x2-1] < q[k]:
                        left += 1
                    goods[k] -= Q[k][x2-1]
                dist += d[path[i2-1]][path[i2+1]][0] \
                      - d[path[i2-1]][x2][0] - d[x2][path[i2+1]][0]
                path = path[:i2] + path[i2+1:]
                pool.append(x2)

                if left <= 0 and dist < ret:
                    ret = dist
                    ret_path = path[:]
                    ret_goods = goods[:]
                    ret_left = left
                    ret_pool = pool[:]

                if num >= 3:
                    for i3 in range(1, len(path)-1):
                        x3 = path[i3]
                        for k in range(n):
                            if goods[k] >= q[k] and goods[k]-Q[k][x3-1] < q[k]:
                                left += 1
                            goods[k] -= Q[k][x3-1]
                        dist += d[path[i3-1]][path[i3+1]][0] \
                              - d[path[i3-1]][x3][0] - d[x3][path[i3+1]][0]
                        path = path[:i3] + path[i3+1:]
                        pool.append(x3)

                        if left <= 0 and dist < ret:
                            ret = dist
                            ret_path = path[:]
                            ret_goods = goods[:]
                            ret_left = left
                            ret_pool = pool[:]

                        path = path[:i3] + [x3] + path[i3:]
                        for k in range(n):
                            if goods[k] < q[k] and goods[k]+Q[k][x3-1] >= q[k]:
                                left -= 1
                            goods[k] += Q[k][x3-1]
                        dist += d[path[i3-1]][x3][0] + d[x3][path[i3+1]][0] \
                              - d[path[i3-1]][path[i3+1]][0]
                        pool.pop()

                path = path[:i2] + [x2] + path[i2:]
                for k in range(n):
                    if goods[k] < q[k] and goods[k]+Q[k][x2-1] >= q[k]:
                        left -= 1
                    goods[k] += Q[k][x2-1]
                dist += d[path[i2-1]][x2][0] + d[x2][path[i2+1]][0] \
                      - d[path[i2-1]][path[i2+1]][0]
                pool.pop()
                
                
        path = path[:i1] + [x1] + path[i1:]
        for k in range(n):
            if goods[k] < q[k] and goods[k]+Q[k][x1-1] >= q[k]:
                left -= 1
            goods[k] += Q[k][x1-1]
        dist += d[path[i1-1]][x1][0] + d[x1][path[i1+1]][0] \
              - d[path[i1-1]][path[i1+1]][0]
        pool.pop()

    return ret, ret_path, ret_goods, ret_left, ret_pool

def switch(path, dist, num):
    global d
    ret = dist
    ret_path = []
    for i1 in range(1, len(path)-1):
        for j1 in range(1, len(path)-1):
            if i1 == j1:
                continue

            if i1 == j1+1 or i1+1 == j1:
                y1 = min(i1, j1)
                y2 = max(i1, j1)
                dist += d[path[y1-1]][path[y2]][0] + d[path[y2]][path[y1]][0] + d[path[y1]][path[y2+1]][0] \
                      - d[path[y1-1]][path[y1]][0] - d[path[y1]][path[y2]][0] - d[path[y2]][path[y2+1]][0]
            else:
                dist += d[path[i1-1]][path[j1]][0] + d[path[j1]][path[i1+1]][0] \
                      + d[path[j1-1]][path[i1]][0] + d[path[i1]][path[j1+1]][0] \
                      - d[path[i1-1]][path[i1]][0] - d[path[i1]][path[i1+1]][0] \
                      - d[path[j1-1]][path[j1]][0] - d[path[j1]][path[j1+1]][0]
            path[i1], path[j1] = path[j1], path[i1]

            if dist < ret:
                ret = dist
                ret_path = path[:]

            if num >= 2:
                for i2 in range(1, len(path)-1):
                    for j2 in range(1, len(path)-1):
                        if i2 == j2:
                            continue

                        if i2 == j2+1 or i2+1 == j2:
                            y1 = min(i2, j2)
                            y2 = max(i2, j2)
                            dist += d[path[y1-1]][path[y2]][0] + d[path[y2]][path[y1]][0] + d[path[y1]][path[y2+1]][0] \
                                  - d[path[y1-1]][path[y1]][0] - d[path[y1]][path[y2]][0] - d[path[y2]][path[y2+1]][0]
                        else:
                            dist += d[path[i2-1]][path[j2]][0] + d[path[j2]][path[i2+1]][0] \
                                + d[path[j2-1]][path[i2]][0] + d[path[i2]][path[j2+1]][0] \
                                - d[path[i2-1]][path[i2]][0] - d[path[i2]][path[i2+1]][0] \
                                - d[path[j2-1]][path[j2]][0] - d[path[j2]][path[j2+1]][0]
                        path[i2], path[j2] = path[j2], path[i2]

                        if dist < ret:
                            ret = dist
                            ret_path = path[:]

                        if num >= 3:
                            for i3 in range(1, len(path)-1):
                                for j3 in range(1, len(path)-1):
                                    if i3 == j3:
                                        continue

                                    if i3 == j3+1 or i3+1 == j3:
                                        y1 = min(i3, j3)
                                        y2 = max(i3, j3)
                                        dist += d[path[y1-1]][path[y2]][0] + d[path[y2]][path[y1]][0] + d[path[y1]][path[y2+1]][0] \
                                            - d[path[y1-1]][path[y1]][0] - d[path[y1]][path[y2]][0] - d[path[y2]][path[y2+1]][0]
                                    else:
                                        dist += d[path[i3-1]][path[j3]][0] + d[path[j3]][path[i3+1]][0] \
                                            + d[path[j3-1]][path[i3]][0] + d[path[i3]][path[j3+1]][0] \
                                            - d[path[i3-1]][path[i3]][0] - d[path[i3]][path[i3+1]][0] \
                                            - d[path[j3-1]][path[j3]][0] - d[path[j3]][path[j3+1]][0]
                                    path[i3], path[j3] = path[j3], path[i3]

                                    if dist < ret:
                                        ret = dist
                                        ret_path = path[:]

                                    if i3 == j3+1 or i3+1 == j3:
                                        y1 = min(i3, j3)
                                        y2 = max(i3, j3)
                                        dist += d[path[y1-1]][path[y2]][0] + d[path[y2]][path[y1]][0] + d[path[y1]][path[y2+1]][0] \
                                            - d[path[y1-1]][path[y1]][0] - d[path[y1]][path[y2]][0] - d[path[y2]][path[y2+1]][0]
                                    else:
                                        dist += d[path[i3-1]][path[j3]][0] + d[path[j3]][path[i3+1]][0] \
                                            + d[path[j3-1]][path[i3]][0] + d[path[i3]][path[j3+1]][0] \
                                            - d[path[i3-1]][path[i3]][0] - d[path[i3]][path[i3+1]][0] \
                                            - d[path[j3-1]][path[j3]][0] - d[path[j3]][path[j3+1]][0]
                                    path[i3], path[j3] = path[j3], path[i3]

                        if i2 == j2+1 or i2+1 == j2:
                            y1 = min(i2, j2)
                            y2 = max(i2, j2)
                            dist += d[path[y1-1]][path[y2]][0] + d[path[y2]][path[y1]][0] + d[path[y1]][path[y2+1]][0] \
                                  - d[path[y1-1]][path[y1]][0] - d[path[y1]][path[y2]][0] - d[path[y2]][path[y2+1]][0]
                        else:
                            dist += d[path[i2-1]][path[j2]][0] + d[path[j2]][path[i2+1]][0] \
                                + d[path[j2-1]][path[i2]][0] + d[path[i2]][path[j2+1]][0] \
                                - d[path[i2-1]][path[i2]][0] - d[path[i2]][path[i2+1]][0] \
                                - d[path[j2-1]][path[j2]][0] - d[path[j2]][path[j2+1]][0]
                        path[i2], path[j2] = path[j2], path[i2]

            if i1 == j1+1 or i1+1 == j1:
                y1 = min(i1, j1)
                y2 = max(i1, j1)
                dist += d[path[y1-1]][path[y2]][0] + d[path[y2]][path[y1]][0] + d[path[y1]][path[y2+1]][0] \
                      - d[path[y1-1]][path[y1]][0] - d[path[y1]][path[y2]][0] - d[path[y2]][path[y2+1]][0]
            else:
                dist += d[path[i1-1]][path[j1]][0] + d[path[j1]][path[i1+1]][0] \
                      + d[path[j1-1]][path[i1]][0] + d[path[i1]][path[j1+1]][0] \
                      - d[path[i1-1]][path[i1]][0] - d[path[i1]][path[i1+1]][0] \
                      - d[path[j1-1]][path[j1]][0] - d[path[j1]][path[j1+1]][0]
            path[i1], path[j1] = path[j1], path[i1]

    return ret, ret_path

def slip(path, dist, num):
    global d
    ret = dist
    ret_path = []
    for i1 in range(1, len(path)-1):
        x1 = path[i1]

        dist += d[path[i1-1]][path[i1+1]][0] \
              - d[path[i1-1]][x1][0] - d[x1][path[i1+1]][0]
        path = path[:i1] + path[i1+1:]
        for j1 in range(1, len(path)):
            dist += d[path[j1-1]][x1][0] + d[x1][path[j1]][0] \
                  - d[path[j1-1]][path[j1]][0]
            path = path[:j1] + [x1] + path[j1:]

            if dist < ret:
                ret = dist
                ret_path = path[:]

            if num >= 2:
                for i2 in range(1, len(path)-1):
                    x2 = path[i2]

                    dist += d[path[i2-1]][path[i2+1]][0] \
                          - d[path[i2-1]][x2][0] - d[x2][path[i2+1]][0]
                    path = path[:i2] + path[i2+1:]
                    for j2 in range(1, len(path)):
                        dist += d[path[j2-1]][x2][0] + d[x2][path[j2]][0] \
                              - d[path[j2-1]][path[j2]][0]
                        path = path[:j2] + [x2] + path[j2:]

                        if dist < ret:
                            ret = dist
                            ret_path = path[:]

                        if num >= 3:
                            for i3 in range(1, len(path)-1):
                                x3 = path[i3]

                                dist += d[path[i3-1]][path[i3+1]][0] \
                                      - d[path[i3-1]][x3][0] - d[x3][path[i3+1]][0]
                                path = path[:i3] + path[i3+1:]
                                for j3 in range(1, len(path)):
                                    dist += d[path[j3-1]][x3][0] + d[x3][path[j3]][0] \
                                          - d[path[j3-1]][path[j3]][0]
                                    path = path[:j3] + [x3] + path[j3:]

                                    if dist < ret:
                                        ret = dist
                                        ret_path = path[:]

                                    dist += d[path[j3-1]][path[j3+1]][0] \
                                          - d[path[j3-1]][x3][0] - d[x3][path[j3+1]][0]
                                    path = path[:j3] + path[j3+1:]
                                
                                path = path[:i3] + [x3] + path[i3:]
                                dist += d[path[i3-1]][x3][0] + d[x3][path[i3+1]][0] \
                                      - d[path[i3-1]][path[i3+1]][0]

                        dist += d[path[j2-1]][path[j2+1]][0] \
                              - d[path[j2-1]][x2][0] - d[x2][path[j2+1]][0]
                        path = path[:j2] + path[j2+1:]
                    
                    path = path[:i2] + [x2] + path[i2:]
                    dist += d[path[i2-1]][x2][0] + d[x2][path[i2+1]][0] \
                          - d[path[i2-1]][path[i2+1]][0]

            dist += d[path[j1-1]][path[j1+1]][0] \
                  - d[path[j1-1]][x1][0] - d[x1][path[j1+1]][0]
            path = path[:j1] + path[j1+1:]
        
        path = path[:i1] + [x1] + path[i1:]
        dist += d[path[i1-1]][x1][0] + d[x1][path[i1+1]][0] \
              - d[path[i1-1]][path[i1+1]][0]
        
    return ret, ret_path

def swap(path, dist, goods, left, pool, num):
    global d, q, Q
    ret = dist
    ret_path = []
    ret_goods = []
    ret_left = left
    ret_pool = []
    for i1 in range(1, len(path)-1):
        for j1 in range(len(pool)):
            for k in range(n):
                if goods[k] < q[k] and goods[k] + Q[k][pool[j1]-1] - Q[k][path[i1]-1] >= q[k]:
                    left -= 1
                elif goods[k] >= q[k] and goods[k] + Q[k][pool[j1]-1] - Q[k][path[i1]-1] < q[k]:
                    left += 1
                goods[k] += Q[k][pool[j1]-1] - Q[k][path[i1]-1]
            dist += d[path[i1-1]][pool[j1]][0] + d[pool[j1]][path[i1+1]][0] \
                  - d[path[i1-1]][path[i1]][0] - d[path[i1]][path[i1+1]][0]
            path[i1], pool[j1] = pool[j1], path[i1]

            if left <= 0 and dist < ret:
                ret = dist
                ret_path = path[:]
                ret_goods = goods[:]
                ret_left = left
                ret_pool = pool[:]

            if num >= 2:
                for i2 in range(1, len(path)-1):
                    for j2 in range(len(pool)):
                        for k in range(n):
                            if goods[k] < q[k] and goods[k] + Q[k][pool[j2]-1] - Q[k][path[i2]-1] >= q[k]:
                                left -= 1
                            elif goods[k] >= q[k] and goods[k] + Q[k][pool[j2]-1] - Q[k][path[i2]-1] < q[k]:
                                left += 1
                            goods[k] += Q[k][pool[j2]-1] - Q[k][path[i2]-1]
                        dist += d[path[i2-1]][pool[j2]][0] + d[pool[j2]][path[i2+1]][0] \
                            - d[path[i2-1]][path[i2]][0] - d[path[i2]][path[i2+1]][0]
                        path[i2], pool[j2] = pool[j2], path[i2]

                        if left <= 0 and dist < ret:
                            ret = dist
                            ret_path = path[:]
                            ret_goods = goods[:]
                            ret_left = left
                            ret_pool = pool[:]

                        if num >= 3:
                            for i3 in range(1, len(path)-1):
                                for j3 in range(len(pool)):
                                    for k in range(n):
                                        if goods[k] < q[k] and goods[k] + Q[k][pool[j3]-1] - Q[k][path[i3]-1] >= q[k]:
                                            left -= 1
                                        elif goods[k] >= q[k] and goods[k] + Q[k][pool[j3]-1] - Q[k][path[i3]-1] < q[k]:
                                            left += 1
                                        goods[k] += Q[k][pool[j3]-1] - Q[k][path[i3]-1]
                                    dist += d[path[i3-1]][pool[j3]][0] + d[pool[j3]][path[i3+1]][0] \
                                        - d[path[i3-1]][path[i3]][0] - d[path[i3]][path[i3+1]][0]
                                    path[i3], pool[j3] = pool[j3], path[i3]

                                    if left <= 0 and dist < ret:
                                        ret = dist
                                        ret_path = path[:]
                                        ret_goods = goods[:]
                                        ret_left = left
                                        ret_pool = pool[:]

                                    for k in range(n):
                                        if goods[k] < q[k] and goods[k] + Q[k][pool[j3]-1] - Q[k][path[i3]-1] >= q[k]:
                                            left -= 1
                                        elif goods[k] >= q[k] and goods[k] + Q[k][pool[j3]-1] - Q[k][path[i3]-1] < q[k]:
                                            left += 1
                                        goods[k] += Q[k][pool[j3]-1] - Q[k][path[i3]-1]
                                    dist += d[path[i3-1]][pool[j3]][0] + d[pool[j3]][path[i3+1]][0] \
                                        - d[path[i3-1]][path[i3]][0] - d[path[i3]][path[i3+1]][0]
                                    path[i3], pool[j3] = pool[j3], path[i3]

                        for k in range(n):
                            if goods[k] < q[k] and goods[k] + Q[k][pool[j2]-1] - Q[k][path[i2]-1] >= q[k]:
                                left -= 1
                            elif goods[k] >= q[k] and goods[k] + Q[k][pool[j2]-1] - Q[k][path[i2]-1] < q[k]:
                                left += 1
                            goods[k] += Q[k][pool[j2]-1] - Q[k][path[i2]-1]
                        dist += d[path[i2-1]][pool[j2]][0] + d[pool[j2]][path[i2+1]][0] \
                            - d[path[i2-1]][path[i2]][0] - d[path[i2]][path[i2+1]][0]
                        path[i2], pool[j2] = pool[j2], path[i2]
            
            for k in range(n):
                if goods[k] < q[k] and goods[k] + Q[k][pool[j1]-1] - Q[k][path[i1]-1] >= q[k]:
                    left -= 1
                elif goods[k] >= q[k] and goods[k] + Q[k][pool[j1]-1] - Q[k][path[i1]-1] < q[k]:
                    left += 1
                goods[k] += Q[k][pool[j1]-1] - Q[k][path[i1]-1]
            dist += d[path[i1-1]][pool[j1]][0] + d[pool[j1]][path[i1+1]][0] \
                  - d[path[i1-1]][path[i1]][0] - d[path[i1]][path[i1+1]][0]
            path[i1], pool[j1] = pool[j1], path[i1]

    return ret, ret_path, ret_goods, ret_left, ret_pool

ext = 3 if m*n <= 200 else 2

def next():
    global curpath, curdist, curgoods, curleft, curpool, end, bestdist, bestpath, ext
    for i in range(1, ext+1):
        dist1, path1, goods1, left1, pool1 = cut(curpath, curdist, curgoods, curleft, curpool, i)
        dist2, path2 = switch(curpath, curdist, i)
        dist3, path3 = slip(curpath, curdist, i)
        dist4, path4, goods4, left4, pool4 = swap(curpath, curdist, curgoods, curleft, curpool, i-1)
        x = min(dist1, dist2, dist3, dist4)
        if x < curdist:
            curdist = x
            if dist1 == x:
                curpath = path1[:]
                curgoods = goods1[:]
                curleft = left1
                curpool = pool1[:]
            elif dist2 == x:
                curpath = path2[:]
            elif dist3 == x:
                curpath = path3[:]
            elif dist4 == x:
                curpath = path4[:]
                curgoods = goods4[:]
                curleft = left4
                curpool = pool4[:]
            break
        if i == ext:
            end = True

end = False

def VNS():
    global bestdist, bestpath
    while True:
        next()
        if end:
            break
    if curdist < bestdist:
        bestdist = curdist
        bestpath = curpath

while r < m:
    initial()
    VNS()
outp()

0 2 4 1 5 0
19


In [ ]:
#simulated annealing

import random
import math

def outp():
    global Path, d, ans
    print(*d[0][Path[0]+1][1], end=' ')
    for i in range(len(Path)-1):
        print(*d[Path[i]+1][Path[i+1]+1][1], end=' ')
    print(*d[Path[len(Path)-1]+1][0][1], end=' ')
    print(0)
    print(ans)

def initial():
    global two_round, m, ans, End, F, Path
    # pool = [i for i in range(m)]
    # random.shuffle(pool)
    # two_round = [0, 0]
    # for i in range(m):
    #     min = 10**10
    #     ir = 0
    #     for j in range(len(two_round)-1):
    #         x = d[two_round[j]][pool[i]+1][0] + d[pool[i]+1][two_round[j+1]][0] \
    #             - d[two_round[i]][two_round[i+1]][0]
    #         if x < min:
    #             ir = j
    #             min = x
    #     two_round.insert(ir+1, pool[i])
    # # print(two_round)
    # # exit(0)
    # two_round = two_round[1:-1]*2
    two_round = [i for i in range(m)]
    random.shuffle(two_round)
    two_round *= 2
    ans, End = value(two_round)
    F = ans
    Path = two_round[:End+1][:]

def value(path):
    global d, n, Q
    it = 0
    left = n
    ret = d[0][path[0]+1][0]
    Qtmp = [0]*n
    for j in range(n):
        if Qtmp[j]+Q[j][path[it]] >= q[j]:
            left -= 1
        Qtmp[j] += Q[j][path[it]]
    while True:
        if it > 0:
            ret += d[path[it-1]+1][path[it]+1][0]
            for j in range(n):
                if Qtmp[j] < q[j] and Qtmp[j]+Q[j][path[it]] >= q[j]:
                    left -= 1
                Qtmp[j] += Q[j][path[it]]
        if left <= 0:
            break
        it += 1
    ret += d[path[it]+1][0][0]
    return ret, it

def better(x, y):
    global T
    if y <= x:
        return True
    else:
        p = random.random()
        # print (y)
        # print(math.exp(-(y-x)/T), p)
        return True if math.exp(-(y-x)/T) >= p else False

cnt = 0
tabu_list = [(0,0)]*m

def next():
    global m, End, two_round, ans, Path, F, cnt, tabu_list
    l = 0
    r = 0
    while True:
        l, r = random.sample(range(m), 2)
        if l <= End or r <= End:
            break
    if r < l:
        r += m
    if (l,r) in tabu_list:
        return
    tabu_list[cnt%m] = (l,r)
    tmp = two_round[:]
    tmp[l:r+1] = tmp[r:(l-1 if l > 0 else None):-1]
    if r < m:
        tmp[l+m:r+m+1] = tmp[l:r+1]
    else:
        tmp[:r-m+1] = tmp[m:r+1]
        tmp[l+m:] = tmp[l:m]
    f, end = value(tmp)
    # if (l,r) in tabu_list:
    #     if f < ans:
    #         ans = f
    #         Path = tmp[:end+1]
    #         F = f
    #         two_round = tmp[:]
    #         End = end
    #         tabu_list[cnt%m] = (l,r)
    #         cnt += 1
    #     return
    if f < ans:
        ans = f
        Path = tmp[:end+1]
    if better(F, f):
        F = f
        two_round = tmp[:]
        End = end
        # tabu_list[cnt%m] = (l,r)
        cnt += 1
    # print(F)
    # print(two_round)
    # print(End)

def anneal():
    global T, cnt
    T = 2000
    cooling_rate = 3e-4
    k_max = 20000
    for k in range(k_max):
        T *= 1-cooling_rate
        next()

initial()
anneal()
outp()

0 2 4 5 1 0
19


In [ ]:
#genetic algorithm
max_generation = 200
crossover_rate = 0.9
mutation_rate = 0.05
pop_size = 2000
def random_indivs(m):
    lst=[]
    for i in range (1,m+1):
        lst.append(i)
    random.shuffle(lst)
    return lst
def check(sum):
  for i in range (len(sum)):
    if sum[i]<q[i]:
      return False
  return True
def calculate_fitness(indivs):
    global end
    sum=[0 for i in range (n)]
    pathcost=d[0][indivs[0]][0]
    for j in range(n):
        sum[j]+=Q[j][indivs[0]-1]
    for i in range (1,m): #len of indivs
        if check(sum):
          end=indivs[i-1]
          break
        pathcost+=d[indivs[i-1]][indivs[i]][0]
        for j in range(n):
            sum[j]+=Q[j][indivs[i]-1]
    pathcost+=d[end][0][0]
    return pathcost

def swap_mutation(indivs):
    p1=random.randint(1,m//2)
    p2=random.randint(1,m//2)
    while (p1==p2):
        p2=random.randint(1,m//2)
    temp=indivs[p1]
    indivs[p1]=indivs[p2]
    indivs[p2]=temp
def random_population(pop_size):
    pop=[]
    while (len(pop) <= pop_size):
        point=random_indivs(m)
        pop.append(point)
    return pop
def single_point_crossover(point1, point2):
    o1=[]
    o2=[]
    v1=[]
    v2=[]
    for i in range (m+10):
        v1.append(0)
        v2.append(0)
    p=random.randint(1,m//2)
    for i in range (p):
        o1.append(point2[i])
        o2.append(point1[i])
        v1[point2[i]]=1
        v2[point1[i]]=1
    index=0
    for i in range (p,m):
        while(v1[point1[index]]!=0):
            index+=1
        o1.append(point1[index])
        v1[o1[i]]=1
    index=0
    for i in range (p,m):
        while(v2[point2[index]]!=0):
            index+=1
        o2.append(point2[index])
        v2[o2[i]]=1
    return [o1,o2]
def comparator(point):
    return (calculate_fitness(point))
def survivor_selection(pop,pop_size):
    pop.sort(key=comparator)
    while (len(pop) > pop_size):
        pop.pop(180)
        # pop.pop()
def reproduction(pop):
    offspring=[]
    while (len(offspring) < pop_size):
        p1=random.randint(0,pop_size/2)
        p2=random.randint(0,pop_size/2)
        while(p1==p2):
            p2=random.randint(0,pop_size/2)
        if(random.randint(1,10)/10 < crossover_rate):
            #print(p1,p2)
            child=single_point_crossover(pop[p1],pop[p2])
            offspring.append(child[0])
            offspring.append(child[1])
    for i in range(len(offspring)):
        if(random.randint(1,100)/100 < mutation_rate):
            swap_mutation(offspring[i])
    return offspring
def print_path(indivs):
    sum=[0 for i in range (n)]
    result=[0,indivs[0]]
    for j in range(n):
        sum[j]+=Q[j][indivs[0]-1]
    for i in range (1,m): #len of indivs
        if check(sum):
            break
        for j in range(n):
            sum[j]+=Q[j][indivs[i]-1]
        result.append(indivs[i])
    result.append(0)
    return(result)
def outp(Path):
    print(*d[0][Path[0]][1], end=' ')
    for i in range(len(Path)-1):
        print(*d[Path[i]][Path[i+1]][1], end=' ')
    print(*d[Path[len(Path)-1]][0][1], end=' ')
    print(0)
def main():
    generation=0
    pop=random_population(pop_size)
    while(generation < max_generation):
        generation+=1
        offspring=reproduction(pop)
        for i in range(len(offspring)):
            pop.append(offspring[i])
        survivor_selection(pop,pop_size)
        best=calculate_fitness(pop[0])
        # print("Generation",generation,"has the best result:",best)
    print("Best result is:",best)
    print("Best path is: ",end='')
    outp(print_path(pop[0])[1:-1])
    return [best,print_path(pop[0])]
# def reinforce():
#   for i in range(10):
#     if(i==0): 
#       k=main()
#       glo=k[0]
#     else:
#       k=main()
#       if(k[0]<glo): 
#         glo=k[0]
#         path=k[1]
#   print(glo)
#   print(*path)
main()
#reinforce()

Best result is: 19
Best path is: 0 2 4 5 1 0


[19, [0, 2, 4, 5, 1, 0]]

In [ ]:
!pip install ortools
import random
import copy
import time
from ortools.sat.python import cp_model

class PickupProb:
    def __init__(self, mode='fast', m=4, n=3):
        if mode == 'fast': # A fast example
            self.m = 4
            self.n = 3
            self.Q = [[0,0,0],[3,0,0],[0,2,0],[0,0,3],[0,1,1]]
            self.d = [[0,2,5,6,6],[2,0,3,4,5],[1,3,0,3,2],[3,4,3,0,6],[4,5,2,6,0]]
            self.q = [3, 3, 3]
        
        if mode == 'random': # Generate a random problem
            self.m = m
            self.n = n
            self.Q = [[0 for i in range(n)] for j in range(m)]
            self.d = [[0 for i in range(m+1)] for j in range(m+1)]
            self.q = [0 for i in range(n)]
            self.Q.insert(0, [0 for i in range(self.n)])

            for i in range(self.m):
                for j in range(self.n):
                    self.Q[i][j] = random.randint(0, 4)
            
            for i in range(self.m+1):
                for j in range(self.m+1):
                    if i == j:
                        self.d[i][j] = 0
                    else:
                        self.d[i][j] = random.randint(2, 9)

            for i in range(self.n):
                self.q[i] = random.randint(10, 15)

        if mode == 'user': # Prompt the user to input the problem
            [self.m, self.n] = [int(i) for i in input('Enter values for n and m separated by a space:').split()]
            
            print('Enter values in Q row by row:')
            self.Q = [[int(i) for i in input().split()] for j in range(self.n)]
            
            print('Enter values for d row by row:')
            self.d = [[int(i) for i in input.split()] for j in range(self.m+1)]

            print('Enter values in q:')
            self.q = [int(i) for i in range(self.n)]
        
        if mode == 'text': # take value from the input.txt
            with open('input.txt') as f:
                al=f.readlines()
            # print(al)
            (self.m, self.n) = al[0].strip().split(' ')
            self.m=int(self.m)
            self.n=int(self.n)
            self.Q=[[]for i in range(self.n)]
            for i in range(self.n):
                k = al[i+1].strip('\n').split(' ')
                for j in range (self.m):
                    self.Q[i].append(int(k[j]))
            #print(Q)
            self.d=[[]for i in range(self.m+1)]
            for i in range(self.m+1):
                k = al[i+self.n+1].strip('\n').split(' ')
                for j in range(self.m+1):
                    self.d[i].append([int(k[j]),[i]])
            self.q=[]
            #print(d)
            k = al[self.m+self.n+2].strip('\n').split(' ')
            for i in range(self.n):
                self.q.append(int(k[i]))
            #print(q)

        # Prepocess
        self.prep = copy.deepcopy(self.d)
        self.path = list(
            list( str(j)+'->' for i in range(self.m+1)) for j in range(self.m+1)
        )

        for k in range(self.m+1):
            for i in range(self.m+1):
                for j in range(self.m+1):
                    if self.prep[i][j] > self.d[i][k] + self.d[k][j]:
                        self.prep[i][j] = self.d[i][k] + self.d[k][j]
                        self.path[i][j] = self.path[i][k] + self.path[k][j]
    
    def printProb(self): # Print the problem
        print('m = ', self.m)
        print('n = ', self.n)
        print('Q = ', self.Q)
        print('d = ', self.d)  
        print('q = ', self.q) 
        print()
        
    def solveCP(self):
        def solveN(num_var):
            model = cp_model.CpModel()

            x = list(
                list(model.NewBoolVar('') for i in range(self.m+1)) for j in range(num_var)
            )
            path = list(
                list(
                    list(
                        model.NewBoolVar('') for i in range(self.m+1)
                    ) for j in range(self.m+1)
                ) for k in range(num_var-1)
            )

            for i in range(num_var):
                model.AddExactlyOne(x[i][j] for j in range(self.m+1))
            
            for i in range(1, self.m+1):
                model.Add(sum(x[j][i] for j in range(num_var)) <= 1)

            for i in range(num_var):
                if i == 0 or i == num_var-1:
                    model.Add(x[i][0] == 1)
                else:
                    model.Add(x[i][0] == 0)

            for i in range(self.n):
                model.Add(sum(sum(self.Q[j][i]*x[k][j] for j in range(self.m+1)) for k in range(num_var)) >= self.q[i])
                
            for k in range(num_var-1):
                for i in range(self.m+1):
                    for j in range(self.m+1):
                        model.Add(x[k][i]+x[k+1][j] >= 2).OnlyEnforceIf(path[k][i][j])
                        model.Add(x[k][i]+x[k+1][j] < 2).OnlyEnforceIf(path[k][i][j].Not())

            objective_terms = []
            for k in range(num_var-1):
                temp = sum(sum(self.prep[i][j]*path[k][i][j] for i in range(self.m+1)) for j in range(self.m+1))
                objective_terms.append(temp)
            model.Minimize(sum(objective_terms))

            solver = cp_model.CpSolver()
            solver.parameters.max_time_in_seconds = 10
            status = solver.Solve(model)
            
            if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
                print(f'Total distance: {solver.ObjectiveValue()}')
                path = ''
                pre = 0
                for i in range(1, num_var):
                    for j in range(self.m+1):
                        if solver.BooleanValue(x[i][j]):
                            path += self.path[pre][j]
                            pre = j
                path = path + '0'
                print('Path: ', path)
                return solver.ObjectiveValue(), path
            else:
                print('No solution found.')
                return float('inf'), None

        best = float('inf')
        best_path = None
        for i in range(3, self.m+3):
            print('Shelves visited:', i-2)
            temp = solveN(i)
            if temp[0] < best:
                best = temp[0]
                best_path = temp[1]
            print('Best so far: ', best, '\n')
        
        if best == float('inf'):
            print('Can\'t find a solution')

        return best, best_path

pr = PickupProb()
pr.printProb()
start = time.perf_counter()
best, best_path = pr.solveCP()
lon = time.perf_counter() - start
print('Best path found: ', best_path)
print('Best path\'s cost: ', best)
print('Done\n')
path_min = [int(i) for i in best_path.split('->')]
print(*path_min)
print(best)
print('Time to solve:', lon*1000, 'ms')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 KB 31.4 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.6
    Uninstalling protobuf-3.19.6:
      Successfully uninstalled protobuf-3.19.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.11.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.22.0 which is incompatible.
tensorflow-metadata 1.12.0 requires protobuf<4,>=3.13, but you have protobuf 4.22.0 which is incompatible.
tensorboard 2.11.2 requires protobuf<4,>=3.9.2, but you have protobuf 4.22.0 which is incompatible.
m =  4
n =  3
Q =  [[0, 0, 0], [3, 0, 0], [0, 2, 0], [0, 0, 3], [0, 1, 1]]
d =  [[0, 2, 5, 6

TypeError: ignored